First of all we need all the necessary librarys to run this beauty!

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from random import randrange, shuffle
from os import path
import librosa
from librosa.display import specshow
import IPython.display as ipd
from math import floor
import kapre
import time

***Check to see if GPU is available

In [2]:
if (len(tf.config.experimental.list_physical_devices("GPU")) > 0):
    print("GPU is available!")
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

GPU is available!


Open and read the csv with all the audio information

In [3]:
csvPath    = r"F:\labeled_audio_data.csv"
specPath   = r"C:\Users\cboen\Documents\Programmierungen\AudioDataScience\Labeled_Specs"
audioPath  = r"F:\Labeled_Audio"
modelPath  = r"F:\Models"
logPath    = r"F:\Logs"

version      = r"3.2" #When you make changes, update the version!!
creationTime = time.strftime("%d.%m.%H.%M.%S", time.localtime())

dataFrame = pd.read_csv(csvPath)
dataDictionary = dataFrame.to_dict()
print(dataDictionary.keys())

sampleRate      = 16000
audioFileLength = 3
spec_hopSize    = 256
spec_blockSize  = 1024
audioFileSize   = (sampleRate * audioFileLength, 1)

dict_keys(['Unnamed: 0', 'filename', 'mainSound', 'length', 'sampleRate', 'quality', 'isCut', 'isMixed', 'isChecked', 'threat', 'salience', 'importance'])


Now just use a snippet of the data for now,
This is to be RAM friendly and for testing purposes.
The Training and Test Data will be a list containing a list that looks like the following:
(filename, mainsound, spectrogram, threat, salience, importance)
filename is a string of the filename
mainsound is a string of the sound category
Spectrogram holds a numpy array of the log spectrogram
threat, salience and importance are single numbers and the output values

In [4]:
numAudiosPerBatch = 1800 #Number of audiofiles to train with per Batch

allFileNames = list(dataDictionary['filename'].values())
#allMainSound = list(dataDictionary['mainSound'].values())
#allThreat = list(dataDictionary['threat'].values())
#allSalience = list(dataDictionary['salience'].values())
#allImportance = list(dataDictionary['importance'].values())

training_data = []

dataIndex = 0
numBatches   = floor(len(allFileNames) / numAudiosPerBatch)
testDataSize = len(allFileNames) % numAudiosPerBatch
print("TrainBatches:", numBatches, ", Testdata:", testDataSize)

ids = list(range(0, len(allFileNames)))
shuffle(ids) #, int(time.strftime("%S", time.localtime()))/60

def createTrainingData():
    #ID list ro iterate through all the data
    global dataIndex
    print("Get new training data: ", int(dataIndex/numAudiosPerBatch), "/", (numBatches-1), sep = "")
    for i in range(numAudiosPerBatch):
        perc = int( 20 * i / numAudiosPerBatch)
        bar = "[" + "=" * (perc) + ">" + "-" * (19-perc) + "]"
        print(bar, end="\r")
        audioFile = ids[i+dataIndex]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            audioFilePath = path.join(audioPath, (filename+".wav"))
            audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0 for i in range(10)]
            targetValue[dataDictionary['threat'][audioFile]] = 1
            training_data.append([filename, mainSound, audioData, targetValue])
        except Exception as e:
            #print(e)
            pass
    dataIndex += numAudiosPerBatch
    print()
    print("Done!")

test_data = []

def createTestData():
    for i in range(len(allFileNames)-testDataSize, len(allFileNames)):
        audioFile = ids[i]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            audioFilePath = path.join(audioPath, (filename+".wav"))
            audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0 for i in range(10)]
            targetValue[dataDictionary['threat'][audioFile]] = 1
        
            test_data.append([filename, mainSound, audioData, targetValue])
        except:
            pass

createTrainingData()
createTestData()
#spec_width  = len(training_data[0][2])
#spec_height = len(training_data[0][2][0])
print(len(training_data[0][2]), sep="|") #spectrogram size:

#del allMainSound, allThreat, allSalience, allImportance, ids

TrainBatches: 2 , Testdata: 12
Get new training data: 0/1


C:\Users\cboen\anaconda3\envs\AudioDataScience39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[===================>]
Done!
48000


***Lets plot a random spectrogram!

audioToPick = randrange(0,len(training_data),1)
print(training_data[audioToPick][0], " : ",  training_data[audioToPick][1])
#plt.imshow(training_data[audioToPick][2], cmap="plasma", aspect="auto")
fig, ax = plt.subplots()
img = specshow(training_data[audioToPick][2], y_axis='log', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")
ipd.Audio(path.join(audioPath, (training_data[audioToPick][0]+".wav")))

***Lets describe the dataset:

In [5]:
def describeTrainingData():
    print("Number of audiofiles:", len(training_data))
    print("Lets get the percentage of each Class:")
    categorys = set(dataDictionary['mainSound'].values())
    trainingDataCategorys = [data[1] for data in training_data] 
    for key in categorys:
        print("Key:", key.ljust(17, " "), ", perc.:", round(trainingDataCategorys.count(key)/len(training_data), 3))
    del categorys, trainingDataCategorys

describeTrainingData()

Number of audiofiles: 1796
Lets get the percentage of each Class:
Key: siren             , perc.: 0.334
Key: animal            , perc.: 0.084
Key: human             , perc.: 0.07
Key: clock_alarm       , perc.: 0.009
Key: interior          , perc.: 0.236
Key: exterior          , perc.: 0.267


In [6]:
train_x=[]
train_y=[]
def sortTrainingData():
    global train_x, train_y
    train_x = [np.pad(data[2], (0, (audioFileSize[0]-len(data[2])))) for data in training_data]
    #train_y = [[data[3], data[4], data[5]] for data in training_data]
    train_y = [data[-1] for data in training_data]
    train_x = np.asarray(train_x).reshape(-1, audioFileSize[0])
    train_y = np.array(train_y)

sortTrainingData()

Now the really fun part!
Create a Convolutional network that can get one spectrogram as input data and outputs three levels for threat, salience and importance!

In [7]:
model = keras.models.Sequential()
print(audioFileSize)
mel_layer = kapre.composed.get_melspectrogram_layer(input_shape=audioFileSize, n_fft=spec_blockSize, hop_length=spec_hopSize, sample_rate=sampleRate, n_mels=64, mel_f_max=8000.0, return_decibel=True, output_data_format='default', name='melspectrogram')
vgg_layer = tf.keras.applications.VGG16(input_shape=(184,64,1),weights=None, classes=10)# 

model.add(mel_layer)
model.add(vgg_layer)

modelName = "stft_vgg16-{}-".format(creationTime) + version
print("Name:", modelName)
print(model.summary())

optimizer = keras.optimizers.SGD(learning_rate = 0.01)
tensorboard = tf.keras.callbacks.TensorBoard(path.join(logPath, modelName), histogram_freq=1)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

(48000, 1)
Name: stft_vgg16-15.03.01.26.27-3.2
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram (Sequential)  (None, 184, 64, 1)        0         
_________________________________________________________________
vgg16 (Functional)           (None, 10)                52511434  
Total params: 52,511,434
Trainable params: 52,511,434
Non-trainable params: 0
_________________________________________________________________
None


Now lets get the first training session done!

In [8]:
for batch in range((numBatches-1)):
    try:
        model.fit(train_x, train_y, validation_split=0.05, epochs=10, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong!")
        print(e)
    training_data = []
    createTrainingData()
    sortTrainingData()

try:
    model.fit(train_x, train_y, validation_split=0.05, epochs=10, callbacks=[tensorboard])
except Exception as e:
    print("Something went wrong!")
    print(e)

Epoch 1/7
51/51 [==============================] - 11s 121ms/step - loss: 0.4645 - accuracy: 0.3175 - val_loss: 0.2617 - val_accuracy: 0.4000
Epoch 2/7
51/51 [==============================] - 4s 77ms/step - loss: 0.2478 - accuracy: 0.3762 - val_loss: 0.2462 - val_accuracy: 0.4611
Epoch 3/7
51/51 [==============================] - 4s 78ms/step - loss: 0.2419 - accuracy: 0.4059 - val_loss: 0.2471 - val_accuracy: 0.3611
Epoch 4/7
51/51 [==============================] - 4s 78ms/step - loss: 0.2374 - accuracy: 0.4134 - val_loss: 0.2385 - val_accuracy: 0.4389
Epoch 5/7
51/51 [==============================] - 4s 77ms/step - loss: 0.2371 - accuracy: 0.4090 - val_loss: 0.2525 - val_accuracy: 0.3444
Epoch 6/7
51/51 [==============================] - 4s 77ms/step - loss: 0.2369 - accuracy: 0.3929 - val_loss: 0.2443 - val_accuracy: 0.4000
Epoch 7/7
51/51 [==============================] - 4s 78ms/step - loss: 0.2328 - accuracy: 0.4127 - val_loss: 0.2357 - val_accuracy: 0.4556
Get new training d

***Lets let the model predict some data!

In [32]:
i = randrange(dataIndex, len(allFileNames), 1)
audioFile = ids[i]
filename = allFileNames[audioFile]
audioFilePath = path.join(audioPath, (filename+".wav"))
audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
audioData = np.pad(audioData, (0, (audioFileSize[0]-len(audioData))))
test_x = np.array(audioData).reshape(-1, audioFileSize[0])
mainSound = dataDictionary['mainSound'][ids[i]]
targetValue = [0 for i in range(10)]
targetValue[dataDictionary['threat'][audioFile]] = 1
test_y = np.array(targetValue)
print("Infos: ", end="")
print( filename, dataDictionary['mainSound'][ids[i]], sep=" | ")
#print(path.join(audioPath, (filename+".wav")))
print("Goal:  ", test_y)
prediction = model(test_x)
print("Result: ", end="")
print([round(val, 2) for val in prediction.numpy()[0]])
#print("max Val:", np.argmax(prediction.numpy()[0]))
ipd.Audio(path.join(audioPath, (filename+".wav")))

Infos: 157866-8-0-6 | siren
Goal:   [0 0 0 0 0 0 0 0 1 0]
Result: [0.07, 0.0, 0.01, 0.0, 0.01, 0.0, 0.01, 0.0, 0.89, 0.0]


In [37]:
training_data = test_data[:]
sortTrainingData()
predict_y = model.predict(train_x)

***Get total errors of the test dataset

Last but not least, lets save the model.

In [38]:
model.evaluate(train_x, train_y)

1/1 [==============================] - 0s 299ms/step - loss: 0.1796 - accuracy: 0.5833


[0.1795700341463089, 0.5833333134651184]

In [39]:
model.save(path.join(modelPath, modelName))

INFO:tensorflow:Assets written to: F:\Models\stft_vgg16-15.03.01.26.27-3.2\assets
